In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Standard plotly imports
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots
import cufflinks
from plotly import tools
import plotly.graph_objs as go#visualization
# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
cufflinks.go_offline(connected=True)

# To interactive buttons
import ipywidgets as widgets
from ipywidgets import interact, interact_manual


In [3]:

df = pd.read_csv("/Users/nicholas/Google Drive/Nick/Jupyter Notebooks/Telco Customer Churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [98]:
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,1,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,1,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,1,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,0,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 21 columns):
customerID          7032 non-null object
gender              7032 non-null int64
SeniorCitizen       7032 non-null int64
Partner             7032 non-null object
Dependents          7032 non-null object
tenure              7032 non-null int64
PhoneService        7032 non-null object
MultipleLines       7032 non-null object
InternetService     7032 non-null object
OnlineSecurity      7032 non-null object
OnlineBackup        7032 non-null object
DeviceProtection    7032 non-null object
TechSupport         7032 non-null object
StreamingTV         7032 non-null object
StreamingMovies     7032 non-null object
Contract            7032 non-null object
PaperlessBilling    7032 non-null object
PaymentMethod       7032 non-null object
MonthlyCharges      7032 non-null float64
TotalCharges        7032 non-null float64
Churn               7032 non-null object
dtypes: float64(2), int64(3), obj

In [100]:
df.isnull().sum().values.sum()

0

In [4]:
df.nunique()

customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), obj

In [226]:
#convert to float type
df["TotalCharges"] = df["TotalCharges"].astype(float)

From the data, I see that a few columns that has the value no internet service and no as well.
I assuume that this means the customeer has no deviceprotection due to the lack of internet service
which is the same meaning as no. Hence, let's convert all no internet service into no as well

In [228]:
columns_to_be_replaced = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']

for i in columns_to_be_replaced : 
    df[i]  = df[i].replace({'No internet service' : 'No'})
    

In [229]:
df.nunique()

customerID          7032
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                72
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         2
OnlineBackup           2
DeviceProtection       2
TechSupport            2
StreamingTV            2
StreamingMovies        2
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1584
TotalCharges        6530
Churn                  2
dtype: int64

In [6]:
#defining all our columns
Id_col     = ['customerID']
metric_col = ["Churn"]
cat_cols   = df.nunique()[df.nunique() <= 4][df.nunique() > 2].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in metric_col]
bool_cols = df.nunique()[df.nunique() == 2].keys().tolist()
bool_cols = [x for x in bool_cols if x not in metric_col]
num_cols   = [x for x in df.columns if x not in cat_cols + metric_col + Id_col + bool_cols]

In [7]:
#defining data
trace = go.Pie(labels=df["Churn"].value_counts().keys().tolist(), 
                values=df["Churn"].value_counts().values.tolist(), 
                marker=dict(colors=['red']), 
                # Seting values to 
                hoverinfo="value"
              )
data = [trace]
#defining layout
layout = go.Layout(title="Churn Distribution")
#defining figure and plotting
fig = go.Figure(data = data,layout = layout)
iplot(fig)

In [8]:
#Separating churn and non churn customers
churn     = df[df["Churn"] == "Yes"]
not_churn = df[df["Churn"] == "No"]

In [9]:
num_cols

['tenure', 'MonthlyCharges', 'TotalCharges']

In [11]:
cat_cols

['MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaymentMethod']

In [10]:
#all categorical columns and thier unique values
for i in cat_cols:
    for j in df[i].unique().tolist():
        print(i + ' unique values are ' + j)

MultipleLines unique values are No phone service
MultipleLines unique values are No
MultipleLines unique values are Yes
InternetService unique values are DSL
InternetService unique values are Fiber optic
InternetService unique values are No
OnlineSecurity unique values are No
OnlineSecurity unique values are Yes
OnlineSecurity unique values are No internet service
OnlineBackup unique values are Yes
OnlineBackup unique values are No
OnlineBackup unique values are No internet service
DeviceProtection unique values are No
DeviceProtection unique values are Yes
DeviceProtection unique values are No internet service
TechSupport unique values are No
TechSupport unique values are Yes
TechSupport unique values are No internet service
StreamingTV unique values are No
StreamingTV unique values are Yes
StreamingTV unique values are No internet service
StreamingMovies unique values are No
StreamingMovies unique values are Yes
StreamingMovies unique values are No internet service
Contract unique va

In [23]:
columns_to_be_replaced = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']

for i in columns_to_be_replaced : 
    df[i]  = df[i].replace({'No internet service' : 'No'})

#defining all our columns
Id_col     = ['customerID']
metric_col = ["Churn"]
cat_cols   = df.nunique()[df.nunique() <= 4][df.nunique() > 2].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in metric_col]
bool_cols = df.nunique()[df.nunique() == 2].keys().tolist()
bool_cols = [x for x in bool_cols if x not in metric_col]
num_cols   = [x for x in df.columns if x not in cat_cols + metric_col + Id_col + bool_cols]

#Separating churn and non churn customers
churn     = df[df["Churn"] == "Yes"]
not_churn = df[df["Churn"] == "No"]

In [14]:
data = {}
for i in cat_cols:
    for j in df[i].unique().tolist():
        data[i + '_' + j] = go.Bar(
    x=['churn','not churn'],
    y=[(churn[i][churn[i] == j].value_counts().values.tolist()[0]/churn[i].value_counts().values.sum())*100,(not_churn[i][not_churn[i] == j].value_counts().values.tolist()[0]/not_churn[i].value_counts().values.sum())*100],
    name = j,width=0.4)
trace = list(data.keys())

            
       

In [15]:
layout = go.Layout(
    title = 'Distribution of Categorical Columns in customer attrition',
    barmode='stack',
    showlegend= False
)

# fig = go.Figure(data=data, layout=layout)
# fig = go.Figure(data=data, layout=layout)
fig = tools.make_subplots(rows=2, cols=2, subplot_titles = ('MultipleLines', 'InternetService', 'Contract', 'PaymentMethod'))
fig.append_trace(data['MultipleLines_No phone service'], 1,1)
fig.append_trace(data['MultipleLines_No'], 1,1)
fig.append_trace(data['MultipleLines_Yes'], 1,1)
fig.append_trace(data['InternetService_DSL'], 1,2)
fig.append_trace(data['InternetService_Fiber optic'], 1,2)
fig.append_trace(data['InternetService_No'], 1,2)
fig.append_trace(data['Contract_Month-to-month'], 2,1)
fig.append_trace(data['Contract_One year'], 2,1)
fig.append_trace(data['Contract_Two year'], 2,1)
fig.append_trace(data['PaymentMethod_Electronic check'], 2,2)
fig.append_trace(data['PaymentMethod_Mailed check'], 2,2)
fig.append_trace(data['PaymentMethod_Bank transfer (automatic)'], 2,2)
fig.append_trace(data['PaymentMethod_Credit card (automatic)'], 2,2)
fig.layout.update(layout)
iplot(fig)

/usr/local/lib/python3.7/site-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



We can see that there are major differences in the distribution of each categorical column between churn and not churn customers. <br>

It is entirely obivous in the columns: <br>

Contract: Majority of churn customers are subscribed to a Month-to-month contract while the not churn customers share an equal distribution. This may indicate a product flaw in the month-to-month subscription. <br>

InternetService: Majority of churn customers are using Fiber optic internet service while the not churn customers share an equal distribution <br>

PaymentMethod: Majority of churn customers are paying using electronic check while the not churn customers share an equal distribution <br>

In [19]:
df['SeniorCitizen'] = df['SeniorCitizen'].astype(str)
#Separating churn and non churn customers
churn     = df[df["Churn"] == "Yes"]
not_churn = df[df["Churn"] == "No"]
    

In [24]:
bool_cols

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'PaperlessBilling']

In [25]:
#defining data
data = {}
for i in bool_cols:
    for j in df[i].unique().tolist():
        data[i + '_' + j] = go.Bar(
    x=['churn','not churn'],
    y=[(churn[i][churn[i] == j].value_counts().values.tolist()[0]/churn[i].value_counts().values.sum())*100,(not_churn[i][not_churn[i] == j].value_counts().values.tolist()[0]/not_churn[i].value_counts().values.sum())*100],
    name = j,width=0.4)
trace = list(data.keys())

#defining layout, fig and plotting
fig = tools.make_subplots(rows=4, cols=3, subplot_titles = ('Partner','SeniorCitizen','Partner', 'Dependents',
       'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamTV','StreamingMovies','PaperlessBilling'))
fig.append_trace(data[trace[0]], 1,1)
fig.append_trace(data[trace[1]], 1,1)
fig.append_trace(data[trace[2]], 1,2)
fig.append_trace(data[trace[3]], 1,2)
fig.append_trace(data[trace[4]], 1,3)
fig.append_trace(data[trace[5]], 1,3)
fig.append_trace(data[trace[6]], 2,1)
fig.append_trace(data[trace[7]], 2,1)
fig.append_trace(data[trace[8]], 2,2)
fig.append_trace(data[trace[9]], 2,2)
fig.append_trace(data[trace[10]], 2,3)
fig.append_trace(data[trace[11]], 2,3)
fig.append_trace(data[trace[12]], 3,1)
fig.append_trace(data[trace[13]], 3,1)
fig.append_trace(data[trace[14]], 3,2)
fig.append_trace(data[trace[15]], 3,2)
fig.append_trace(data[trace[16]], 3,3)
fig.append_trace(data[trace[17]], 3,3)
fig.append_trace(data[trace[18]], 4,1)
fig.append_trace(data[trace[19]], 4,1)
fig.append_trace(data[trace[20]], 4,2)
fig.append_trace(data[trace[21]], 4,2)
fig.append_trace(data[trace[22]], 4,3)
fig.append_trace(data[trace[23]], 4,3)
fig.layout.update(layout)
iplot(fig)


Right away we can see that some columns have more obvious differences in their distributions between churn and not churn customers.<br> The obvious ones are <br>

Partner: Majority of churn customers are single while not churn customers have an equal distribution <br>

Dependents: Majority of churn customers does not have dependents, a similar distribution applies for not churn customers but the magnitude is lesser. This may just indicate that the majority of the telco's customers do not have dependents <br>

OnlineSecuirity: Majority of churn customers do not have online secuirity, a similar distribution applies for not churn customers but the magnitude is lesser. This may just indicate that the majority of the telco's customers do not have online secuirity. On the other hand, it makes sense for someone who does not have internet secuirity to have a higher probability to churn because he/she does not prioritize online activity thus the lack of online secuirity. <br>

OnlineBackup: Majority of churn customers do not have online secuirity, a similar distribution applies for not churn customers but the magnitude is lesser. This may just indicate that the majority of the telco's customers do not have online backup. However, just like online secuirity I suspect that customers who do not prioritize online activity do not subscribe to online services like online backup. Hence, they are more likely to churn. I will attempt to run a hypothesis test on this later on. <br>

TechSupport: Majority of churn customers do not have tech support from the telco, a similar distribution applies for not churn customers but the magnitude is lesser. This may just indicate that the majority of the telco's cusomters do not have tech support. However, it makes sense for a customer who does not have tech support to have a higher probability to churn due to the lack of support.  <br>

PaperlessBilling: Majority of churn customers have paperless billing while not churn customers have an equal distribution. This may be a flaw in the paperless billing feature from the telco. <br>

Let's explore our numerical columns now, taking a look at their distribution.

In [243]:
num_cols

['tenure', 'MonthlyCharges', 'TotalCharges']

In [55]:
df['tenure']

0        1
1       34
2        2
3       45
4        2
        ..
7027    24
7028    72
7029    11
7030     4
7031    66
Name: tenure, Length: 7032, dtype: int64

In [245]:
#defining data
trace1 = go.Histogram(x  = churn['tenure'],
                          histnorm= "percent",
                          name = "Churn Customers",
                          marker = dict(line = dict(width = .5,
                                                    color = "black"
                                                    )
                                        ),
                         opacity = 1
                         ) 
    
trace2 = go.Histogram(x  = not_churn['tenure'],
                      histnorm = "percent",
                      name = "Non churn customers",
                      marker = dict(line = dict(width = .5,
                                          color = "black"
                                         )
                             ),
                      opacity = .7
                     )
data = [trace1,trace2]

#defining layout
layout = go.Layout(dict(title ='tenure' + " distribution in customer attrition ",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         title = 'tenure',
                                         zerolinewidth=1,
                                         ticklen=5,
                                         gridwidth=2
                                        ),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         title = "percent",
                                         zerolinewidth=1,
                                         ticklen=5,
                                         gridwidth=2
                                        )
                       ),barmode = 'overlay'
                  )

#defining figure and plotting
fig  = go.Figure(data=data,layout=layout)
iplot(fig)

# fig = go.Figure()
# fig.add_trace(go.Histogram(x=churn['tenure']))
# fig.add_trace(go.Histogram(x=not_churn['tenure']))
# # Overlay both histograms
# # fig.update_layout(barmode='overlay')
# # Reduce opacity to see both histograms
# fig.update_traces(opacity=0.75)
# fig.show()

Exploring the tenure distribution, we see that there is a skewed distribution for churn customers.
<br>About 40% of churn customers have a 1-5 month tenure with the telco.<br>
The distribution for non-churn customers is fairly uniform.


In [246]:
#defining data
trace1 = go.Histogram(x  = churn['MonthlyCharges'],
                          histnorm= "percent",
                          name = "Churn Customers",
                          marker = dict(line = dict(width = .5,
                                                    color = "black"
                                                    )
                                        ),
                         opacity = 1
                         ) 


trace2 = go.Histogram(x  = not_churn['MonthlyCharges'],
                      histnorm = "percent",
                      name = "Non churn customers",
                      marker = dict(line = dict(width = .5,
                                          color = "black"
                                         )
                             ),
                      opacity = .7
                     )
data = [trace1,trace2]

#defining layout
layout = go.Layout(dict(title ='MonthlyCharges' + " distribution in customer attrition ",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         title = 'monthlycharges',
                                         zerolinewidth=1,
                                         ticklen=5,
                                         gridwidth=1
                                        ),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         title = "monthlycharges",
                                         zerolinewidth=1,
                                         ticklen=5,
                                         gridwidth=1
                                        )
                       ),barmode = 'overlay'
                  )

#defining figure and plotting
fig  = go.Figure(data=data,layout=layout)
iplot(fig)

# fig = go.Figure()
# fig.add_trace(go.Histogram(x=churn['tenure']))
# fig.add_trace(go.Histogram(x=not_churn['tenure']))
# # Overlay both histograms
# # fig.update_layout(barmode='overlay')
# # Reduce opacity to see both histograms
# fig.update_traces(opacity=0.75)
# fig.show()

Exploring the monthlycharges distribution, we can see that a majority of churned customers spent between 70 and 105.
A majority(about 30%) of non churn customers spent between 15 and 30.

In [58]:
#defining data
trace1 = go.Histogram(x  = churn['TotalCharges'],
                          histnorm= "percent",
                          name = "Churn Customers",
                          marker = dict(line = dict(width = .5,
                                                    color = "black"
                                                    )
                                        ),
                         opacity = 1
                         ) 


trace2 = go.Histogram(x  = not_churn['TotalCharges'],
                      histnorm = "percent",
                      name = "Non churn customers",
                      marker = dict(line = dict(width = .5,
                                          color = "black"
                                         )
                             ),
                      opacity = .7
                     )
data = [trace1,trace2]

#defining layout
layout = go.Layout(dict(title ='TotalCharges' + " distribution in customer attrition ",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         title = 'TotalCharges',
                                         zerolinewidth=1,
                                         ticklen=5,
                                         gridwidth=1
                                        ),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         title = "TotalCharges",
                                         zerolinewidth=1,
                                         ticklen=5,
                                         gridwidth=1
                                        )
                       ),barmode = 'overlay'
                  )

#defining figure and plotting
fig  = go.Figure(data=data,layout=layout)
iplot(fig)

# fig = go.Figure()
# fig.add_trace(go.Histogram(x=churn['tenure']))
# fig.add_trace(go.Histogram(x=not_churn['tenure']))
# # Overlay both histograms
# # fig.update_layout(barmode='overlay')
# # Reduce opacity to see both histograms
# fig.update_traces(opacity=0.75)
# fig.show()

Exploring the TotalCharges distribution, about 40% of churn customers spend less than 400.
<br>The distribution of non-churn customers is quite uniform having about 50% spending less than 1800.

It would definitely make sense for churn customers to have a lower total charge because they had left the company. 
But what about the monthly charge? We can take a look at that.
At the same time, we can see if there's a relationship between monthly charge and tenure.

In [250]:
#defining data
trace1 = go.Scatter(x=churn['MonthlyCharges'],y=churn['tenure'],mode = "markers",marker = dict(line = dict(color = "blue"),
                                                                   size=churn['TotalCharges']/180,
                                                        color = 'blue',
                                                      
                                                      ),
                        opacity = .9
                       ,name='churn',showlegend=True)

trace2 = go.Scatter(x=not_churn['MonthlyCharges'],y=not_churn['tenure'],mode = "markers",marker = dict(line = dict(color = "red"),
                                                                   size=not_churn['TotalCharges']/180,
                                                       color = 'red',
                                                      
                                                      ),
                        opacity = 0.5
                       ,name='not_churn',showlegend=True)
data = [trace1, trace2]
                       
#defining layout
layout = go.Layout(dict(title = 'Monthly Charges vs Tenure (radius = total charges)',
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         title = "monthly charges",
                                         zerolinewidth=1,ticklen=5,gridwidth=2),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         title = "tenure",
                                         zerolinewidth=1,ticklen=5,gridwidth=2),
                            height = 600
                           )
                      )

#defining figure and plotting
fig = go.Figure(data,layout=layout)
iplot(fig)